In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/")

### Primero, cargamos el dataframe de entrenamiento con la limpieza ya realizada en la entrega numero 2.

In [ ]:
df= pd.read_csv('clean_train.csv')

In [ ]:
df = pd.get_dummies(df, columns=['FAMI_EDUCACIONPADRE','FAMI_EDUCACIONMADRE','FAMI_ESTRATOVIVIENDA'])

### Para este entrenamiento, solo podemos utilizar columnas con valores numericos o booleanos. Para esto nos deshacemos de las columnas que no cumplen estos requisitos

In [ ]:
df = df.drop(columns=['PERIODO', 'ESTU_PRGM_ACADEMICO'])

### Luego mapeamos los valores de la columna RENDIMIENTO_GLOBAL con valores del 0 al 3, esto necesario para poder realizar el entrenamiento porque, como ya se mencionó anteriormente, el modelo solo acepta valores numericos o booleanos.

In [ ]:
# Crear un diccionario de mapeo
mapeo = {
    'bajo': 0,
    'medio-bajo': 1,
    'medio-alto': 2,
    'alto': 3
}

# Aplicar el mapeo a la columna
df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(mapeo)

### Procedemos entonces a realizar el entrenamiento.
El modelo que mejor resultados lanzó fue el de xgboost.

se definen los parametros para la configuración del modelo.
usamos rondas de impulso de 150, usamos método de aproximación y lo definimos para tener una clasificación binaria con una salida de probabilidad.

Luego definimos los dataFrame para proceder con el entrenamiento del modelo, el X_train con todas las caracteristicas exceptuando la columna a predecir y el y_train que contiene solo la columna a predecir. inicializamos el modelo con el impulso de ronda que ya definimos y los parametros para proceder a entrenarlo mandando X_train y y_train como parámetros.

In [ ]:
params = {
    'tree_method': 'approx',
    'objective': 'binary:logistic',
}

num_boost_round = 150

X_train = df.drop(columns=['RENDIMIENTO_GLOBAL'])
y_train = df['RENDIMIENTO_GLOBAL']

clf = xgb.XGBClassifier(n_estimators=num_boost_round, **params)
clf.fit(X_train, y_train)



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=150, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

### Ahora cargamos el dataFrame para testear la predicción del modelo. Procedemos tambien a eliminar las mismas columnas que borramos en el dataFrame de entrenamiento.

In [ ]:
new_df = pd.read_csv('clean_test.csv')


In [ ]:
new_df = pd.get_dummies(new_df, columns=['FAMI_EDUCACIONPADRE','FAMI_EDUCACIONMADRE','FAMI_ESTRATOVIVIENDA'])

In [ ]:
new_df = new_df.drop(columns=['PERIODO', 'ESTU_PRGM_ACADEMICO'])

#### Realizamos la predicción enviando como parametro el dataFrame de test y la almacenamos en una variable.

In [ ]:
predictions = clf.predict(new_df)

### Le añadimos una nueva columna al dataFrame de test donde estan contenidos los valores que predecimos. Hacemos el mapeo inverso para obtener de los numeros los valores en String de las predicciones.

In [ ]:
new_df['RENDIMIENTO_GLOBAL'] = predictions

In [ ]:
mapeo_original = {
    0: 'bajo',
    1: 'medio-bajo',
    2: 'medio-alto',
    3: 'alto'
}

new_df['RENDIMIENTO_GLOBAL'] = new_df['RENDIMIENTO_GLOBAL'].map(mapeo_original)


### Procedemos a almacenar el id junto a la predicción en un nuevo dataFrame para descargarlo y subirlo a Kaggle.

In [ ]:
df_kaggle = new_df.loc[:, ['ID', 'RENDIMIENTO_GLOBAL']]
df_kaggle

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,medio-bajo
3,782980,bajo
4,785185,bajo
...,...,...
296781,496981,medio-bajo
296782,209415,alto
296783,239074,medio-alto
296784,963852,alto


In [ ]:
df_kaggle = df_kaggle.rename(columns={'PREDICCION':'RENDIMIENTO_GLOBAL'})

In [ ]:
df_kaggle


,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,medio-bajo
3,782980,bajo
4,785185,bajo
...,...,...
296781,496981,medio-bajo
296782,209415,alto
296783,239074,medio-alto
296784,963852,alto


In [ ]:
df_kaggle.to_csv('prediction_with_XGBoost.csv',index=False)
from google.colab import files
files.download('prediction_with_XGBoost.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>